# Kurdish Full Alphabet OCR - Advanced Testing Pipeline

## Overview
This notebook implements the inference pipeline for the **Advanced Kurdish OCR Model** (ResNet-Style CNN). It is designed to load the high-accuracy model (approx 95.8%) trained in the upgraded training notebook and test it on new data.

##  Important Implementation Details
Thispipeline requires specific handling:
1.  **Architecture Matching:** We must redefine the exact `KurdishAdvancedCNN` class with Projection Layers to match the saved weights.
2.  **Preprocessing:** Images must be normalized using `mean=[0.456]` and `std=[0.224]` (matching ImageNet stats) rather than simple division.
3.  **Label Recovery:** Class labels are extracted directly from the checkpoint file to ensure 100% alignment with training.

## Contents
1.  **Setup & Imports:** Configure device (GPU/CPU) and libraries.
2.  **Model Definition:** Re-create the Advanced CNN architecture.
3.  **Checkpoint Loading:** Smart loader that retrieves weights + label encoders.
4.  **Inference Engine:** Prediction function with correct normalization.
5.  **Live Testing:** Real-time camera or batch image testing.

In [1]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
from imutils import paths
import os

# Configuration
IMG_SIZE = 64
# This must match the filename saved in the training notebook
MODEL_PATH = "kurdish_letter_model_pytorch.pth" 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## 1. Define Model Architecture
**Crucial Step:** We must define the neural network structure **exactly** as it was during training.
This includes:
* 4 Residual Blocks with Batch Normalization.
* **Projection Shortcuts** (1x1 Convs) to handle channel dimension changes (32 $\to$ 64 $\to$ 128 $\to$ 256).
* **Adaptive Average Pooling** to handle various input sizes without crashing.

In [2]:
class KurdishAdvancedCNN(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.3):
        super(KurdishAdvancedCNN, self).__init__()
        
        # Block 1: Input -> 32
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d(dropout_rate)
        
        # Block 2: 32 -> 64 (With Projection)
        self.project2 = nn.Sequential(nn.Conv2d(32, 64, 1, bias=False), nn.BatchNorm2d(64))
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout2d(dropout_rate)
        
        # Block 3: 64 -> 128 (With Projection)
        self.project3 = nn.Sequential(nn.Conv2d(64, 128, 1, bias=False), nn.BatchNorm2d(128))
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout3 = nn.Dropout2d(dropout_rate)
        
        # Block 4: 128 -> 256 (With Projection)
        self.project4 = nn.Sequential(nn.Conv2d(128, 256, 1, bias=False), nn.BatchNorm2d(256))
        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn7 = nn.BatchNorm2d(256)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn8 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.dropout4 = nn.Dropout2d(dropout_rate)
        
        # Head
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(256, 512)
        self.bn_fc1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn_fc2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, num_classes)
        self.dropout_fc = nn.Dropout(dropout_rate)
        self.leaky_relu = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.leaky_relu(self.bn1(self.conv1(x)))
        res = x
        x = self.leaky_relu(self.bn2(self.conv2(x)))
        x = self.leaky_relu(x + res)
        x = self.pool1(self.dropout1(x))
        
        res = self.project2(x)
        x = self.leaky_relu(self.bn3(self.conv3(x)))
        x = self.bn4(self.conv4(x))
        x = self.leaky_relu(x + res)
        x = self.pool2(self.dropout2(x))
        
        res = self.project3(x)
        x = self.leaky_relu(self.bn5(self.conv5(x)))
        x = self.bn6(self.conv6(x))
        x = self.leaky_relu(x + res)
        x = self.pool3(self.dropout3(x))
        
        res = self.project4(x)
        x = self.leaky_relu(self.bn7(self.conv7(x)))
        x = self.bn8(self.conv8(x))
        x = self.leaky_relu(x + res)
        x = self.pool4(self.dropout4(x))
        
        x = self.global_avg_pool(x).view(x.size(0), -1)
        x = self.leaky_relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout_fc(x)
        x = self.leaky_relu(self.bn_fc2(self.fc2(x)))
        x = self.dropout_fc(x)
        x = self.fc3(x)
        return x

## 2. Load "Gold Standard" Checkpoint
This function loads `kurdish_letter_model_pytorch.pth`.
* **Automatic Class Detection:** It reads the `label_encoder` saved inside the file, so you don't need a separate `.pkl` file.
* **Weights Loading:** It maps the trained weights to the architecture defined above.

In [3]:
def load_trained_model(model_path):
    print(f"Loading checkpoint from: {model_path}")
    
    # FIX: Set weights_only=False to allow loading the LabelEncoder object
    checkpoint = torch.load(model_path, map_location=device, weights_only=False)
    
    # 1. Retrieve Label Encoder
    if 'label_encoder' in checkpoint:
        le = checkpoint['label_encoder']
        classes = le.classes_
        print(f"✅ Found {len(classes)} classes in checkpoint.")
    else:
        # Fallback if you used an older training loop
        raise ValueError("Label encoder not found in checkpoint! Please re-train with the upgraded notebook.")
        
    # 2. Initialize Model with correct class count
    model = KurdishAdvancedCNN(num_classes=len(classes)).to(device)
    
    # 3. Load Weights
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint) # Fallback for older saves
        
    model.eval() # Set to evaluation mode
    print(f"✅ Model loaded successfully! (Trained for {checkpoint.get('epoch', '?')} epochs)")
    print(f"   Validation Accuracy: {checkpoint.get('val_accuracy', 0):.2f}%")
    
    return model, le

# Execute load
try:
    model, label_encoder = load_trained_model(MODEL_PATH)
except FileNotFoundError:
    print(f"❌ Error: {MODEL_PATH} not found. Please run training first!")

Loading checkpoint from: kurdish_letter_model_pytorch.pth
✅ Found 33 classes in checkpoint.
✅ Model loaded successfully! (Trained for 36 epochs)
   Validation Accuracy: 95.87%


## 3. Inference & Preprocessing
The `predict_image` function handles the transition from a raw image to a model tensor.

**Preprocessing Pipeline:**
1.  **Resize:** Force image to 64x64.
2.  **Channel Expansion:** Convert Grayscale $\to$ 3-Channel (Model expects 3 inputs).
3.  **Normalization:** Apply the specific formula: `(pixel - 0.456) / 0.224`.

In [4]:
def predict_image(image_path, model, label_encoder):
    # 1. Read Image
    if isinstance(image_path, str):
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    else:
        image = image_path # Assume it's already a numpy array (e.g. from camera)
        
    if image is None:
        return "Error reading image", 0.0

    # 2. Preprocessing (MUST MATCH TRAINING)
    # Resize
    img = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    
    # Stack to 3 Channels (Because model expects 3)
    img = np.stack([img] * 3, axis=-1)
    
    # Normalize & Standardize
    # (pixel / 255.0 - mean) / std
    img = img.astype('float32') / 255.0
    img = (img - 0.456) / 0.224
    
    # Convert to Tensor (CHW format)
    img = np.transpose(img, (2, 0, 1))
    img_tensor = torch.from_numpy(img).float().unsqueeze(0).to(device)
    
    # 3. Inference
    with torch.no_grad():
        outputs = model(img_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        
        # Get Top Prediction
        prob, predicted_idx = torch.max(probabilities, 1)
        confidence = prob.item() * 100
        predicted_label = label_encoder.inverse_transform([predicted_idx.item()])[0]
        
    return predicted_label, confidence, image

# Test on a random file (if you have one)
# label, conf, _ = predict_image("path/to/test.jpg", model, label_encoder)
# print(f"Prediction: {label} ({conf:.1f}%)")

## 🎥 Live Camera GUI – Advanced Kurdish OCR

This interactive tool provides a real-time feed from your webcam to test the model in a live environment.

### How it Works (Advanced Pipeline)
When you click **Capture ROI**, the system performs the following "Gold Standard" preprocessing steps to match the training data exactly:
1.  **ROI Extraction:** Crops the center square from the video feed.
2.  **3-Channel Conversion:** Converts the grayscale input into a 3-channel format (RGB/BGR) as required by the ResNet architecture.
3.  **Normalization:** Applies ImageNet statistics (`mean=0.456`, `std=0.224`) to center the pixel data.
4.  **Inference:** The processed tensor is fed into the **Advanced CNN**, which outputs the predicted class and confidence score.

### Controls
* **Camera:** Select your camera input source (Index 0, 1, or 2).
* **Start:** Initializes the camera thread and begins the live preview.
* **Stop:** Safely releases the camera and stops the thread.
* **Capture ROI:** Freezes the current frame, processes the region inside the box, and runs the prediction.

### Tips for Best Results
* **Contrast:** Use a **black marker on white paper**. The model is trained on high-contrast character data.
* **Alignment:** Center the letter inside the **Blue/Orange Box**.
* **Lighting:** Ensure the paper is well-lit to avoid shadows, which can be mistaken for strokes.

In [5]:
## Interactive GUI: Camera ROI Capture and Prediction (robust, with diagnostics)

# Enhancements:
# - Camera index selector (0/1/2) and resolution hints
# - Immediate single-frame test after start to confirm preview
# - Heartbeat indicator updated from the preview loop
# - Defensive checks and clear status messages
# - Stop always releases camera; Start prevents duplicate threads

import cv2
import numpy as np
import time
import threading
from IPython.display import display
import ipywidgets as widgets
import torch

# --- BACKEND UPGRADES FOR ADVANCED CNN ---

# Ensure variable compatibility (The GUI uses 'le', our loader uses 'label_encoder')
le = label_encoder 

# Helper: predict from a grayscale numpy array (already 2D)
def predict_from_array(gray_array):
    assert len(gray_array.shape) == 2, "Expected grayscale 2D array"
    
    # 1. Resize to match Model Input (64x64)
    img_resized = cv2.resize(gray_array, (IMG_SIZE, IMG_SIZE))
    
    # 2. Convert to Float and Scale [0, 1]
    img_float = img_resized.astype('float32') / 255.0
    
    # 3. Stack to 3 Channels (Model expects RGB/BGR depth, even if grayscale)
    img_rgb = np.stack([img_float] * 3, axis=-1)
    
    # 4. Normalize using Training Stats (ImageNet Mean/Std)
    # This is CRITICAL for the Advanced ResNet model
    img_norm = (img_rgb - 0.456) / 0.224
    
    # 5. Convert to Tensor (CHW format)
    # Permute moves channel to first dimension: (64, 64, 3) -> (3, 64, 64)
    img_tensor = torch.from_numpy(img_norm).permute(2, 0, 1).unsqueeze(0).float().to(device)
    
    # 6. Inference
    with torch.no_grad():
        output = model(img_tensor)
        probabilities = torch.softmax(output, dim=1)[0].cpu().numpy()
        predicted_idx = int(torch.argmax(output, dim=1).item())
        confidence = float(probabilities[predicted_idx] * 100)
    
    predicted_class = le.classes_[predicted_idx]
    return predicted_class, confidence, probabilities

# --- GUI DEFINITION (UNCHANGED) ---

# Widgets
cam_index_dropdown = widgets.Dropdown(options=[0,1,2], value=0, description='Camera', layout=widgets.Layout(width='180px'))
start_button = widgets.Button(description="Start", button_style="success")
stop_button = widgets.Button(description="Stop", button_style="warning")
capture_button = widgets.Button(description="Capture ROI", button_style="primary")
status_label = widgets.HTML(value="<b>Status:</b> Ready")
heartbeat_label = widgets.HTML(value="<b>Heartbeat:</b> idle")
output_area = widgets.Output()
preview_area = widgets.VBox([])
image_widget = widgets.Image(format='png')
preview_area.children = [image_widget]

controls = widgets.HBox([cam_index_dropdown, start_button, stop_button, capture_button])
ui = widgets.VBox([
    widgets.HTML("""
    <div style='font-family:Inter,Helvetica,Arial,sans-serif; padding:8px 0;'>
      <h2 style='margin:0 0 8px;'>Kurdish Full Alphabet OCR – Live Capture</h2>
      <p style='color:#555; margin:0;'>Place a white paper with a black Kurdish letter within the ROI, then press <b>Capture ROI</b>.</p>
    </div>
    """),
    controls,
    status_label,
    heartbeat_label,
    preview_area,
    output_area
])

display(ui)

# Camera and ROI config
cap = None
running = False
preview_thread = None
last_frame = None
roi = None  # (x,y,w,h)
COLOR_ROI = (0,200,255)

# Frame overlay
def draw_overlay(frame):
    h, w = frame.shape[:2]
    global roi
    if roi is None:
        rw = int(w * 0.6); rh = int(h * 0.6)
        rx = (w - rw) // 2; ry = (h - rh) // 2
        roi = (rx, ry, rw, rh)
    rx, ry, rw, rh = roi
    overlay = frame.copy()
    mask = np.zeros_like(frame)
    cv2.rectangle(mask, (0,0), (w,h), (0,0,0), -1)
    cv2.rectangle(mask, (rx, ry), (rx+rw, ry+rh), (0,0,0), -1)
    alpha = 0.35
    cv2.addWeighted(mask, alpha, overlay, 1-alpha, 0, overlay)
    frame = overlay
    cv2.rectangle(frame, (rx, ry), (rx+rw, ry+rh), COLOR_ROI, 2)
    cv2.putText(frame, 'Align paper within ROI', (rx+10, max(30, ry-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COLOR_ROI, 2, cv2.LINE_AA)
    return frame

# Binarize to white background / black text
def to_binary(gray):
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    if np.mean(thresh) < 127:
        thresh = 255 - thresh
    return thresh

# Preview loop
def preview_loop():
    global cap, running, last_frame
    frame_count = 0
    while running and cap is not None and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            status_label.value = "<b>Status:</b> Camera read failed"
            break
        target_w = 800
        target_h = int(frame.shape[0] * target_w / frame.shape[1])
        frame = cv2.resize(frame, (target_w, target_h))
        last_frame = frame.copy()
        frame_disp = draw_overlay(frame.copy())
        rgb = cv2.cvtColor(frame_disp, cv2.COLOR_BGR2RGB)
        ok, buf = cv2.imencode('.png', rgb)
        if ok:
            image_widget.value = buf.tobytes()
        frame_count += 1
        if frame_count % 15 == 0:
            heartbeat_label.value = f"<b>Heartbeat:</b> {frame_count} frames"
        time.sleep(0.02)
    heartbeat_label.value = "<b>Heartbeat:</b> stopped"

# Event handlers
def on_start_clicked(_):
    global cap, running, preview_thread, last_frame
    if running:
        status_label.value = "<b>Status:</b> Preview already running"
        return
    cam_idx = cam_index_dropdown.value
    cap = cv2.VideoCapture(cam_idx)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    if not cap.isOpened():
        status_label.value = f"<b>Status:</b> Could not open camera index {cam_idx}"
        cap = None
        return
    ret, frame = cap.read()
    if not ret or frame is None:
        status_label.value = "<b>Status:</b> Camera delivered no frames"
        cap.release(); cap = None
        return
    target_w = 800
    target_h = int(frame.shape[0] * target_w / frame.shape[1])
    frame = cv2.resize(frame, (target_w, target_h))
    last_frame = frame.copy()
    frame_disp = draw_overlay(frame.copy())
    rgb = cv2.cvtColor(frame_disp, cv2.COLOR_BGR2RGB)
    ok, buf = cv2.imencode('.png', rgb)
    if ok:
        image_widget.value = buf.tobytes()
    status_label.value = "<b>Status:</b> Camera started"
    heartbeat_label.value = "<b>Heartbeat:</b> starting"
    running = True
    preview_thread = threading.Thread(target=preview_loop)
    preview_thread.daemon = True
    preview_thread.start()

def on_stop_clicked(_):
    global cap, running, preview_thread
    running = False
    if cap:
        cap.release()
        cap = None
    status_label.value = "<b>Status:</b> Camera stopped"
    image_widget.value = b''  # Clear image

def on_capture_clicked(_):
    if not last_frame is not None:
        status_label.value = "<b>Status:</b> No frame to capture"
        return
    
    if roi is None:
        status_label.value = "<b>Status:</b> ROI not defined"
        return
    
    rx, ry, rw, rh = roi
    roi_img = last_frame[ry:ry+rh, rx:rx+rw]
    gray_roi = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)
    
    # Display ROI and its binary version side by side
    binary = to_binary(gray_roi)
    combined = np.hstack([cv2.cvtColor(gray_roi, cv2.COLOR_GRAY2BGR), cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)])
    
    # Predict
    pred_class, confidence, probs = predict_from_array(gray_roi)
    
    # Show results
    with output_area:
        output_area.clear_output(wait=True)
        
        # Convert to RGB for display in Jupyter
        combined_rgb = cv2.cvtColor(combined, cv2.COLOR_BGR2RGB)
        _, buf = cv2.imencode('.png', combined_rgb)
        
        print(f"Predicted: {pred_class}")
        print(f"Confidence: {confidence:.2f}%")
        
        # Show top 5 predictions
        top_5_idx = np.argsort(probs)[::-1][:5]
        print("Top 5 predictions:")
        for idx in top_5_idx:
            class_name = le.classes_[idx]
            prob = probs[idx] * 100
            print(f"  {class_name}: {prob:.2f}%")

start_button.on_click(on_start_clicked)
stop_button.on_click(on_stop_clicked)
capture_button.on_click(on_capture_clicked)

In [6]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def evaluate_model_comprehensive(model, loader, device, label_encoder, top_k=3):
    """
    Runs a full evaluation suite: Confusion Matrix, Classification Report, 
    Top-K Accuracy, and Error Analysis.
    """
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    # Store images for error visualization
    error_images = []
    error_true = []
    error_pred = []
    error_conf = []
    
    print(f"📊 Running Comprehensive Evaluation on {len(loader.dataset)} images...")
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            
            # Get Top-1 prediction
            _, preds = torch.max(outputs, 1)
            
            # Store data
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            
            # Capture Errors
            incorrect_mask = preds != labels
            if incorrect_mask.any():
                bad_imgs = inputs[incorrect_mask].cpu()
                bad_preds = preds[incorrect_mask].cpu()
                bad_true = labels[incorrect_mask].cpu()
                bad_conf = torch.max(probs, dim=1)[0][incorrect_mask].cpu()
                
                for i in range(len(bad_imgs)):
                    if len(error_images) < 20: # Limit to saving 20 errors
                        error_images.append(bad_imgs[i])
                        error_true.append(bad_true[i].item())
                        error_pred.append(bad_preds[i].item())
                        error_conf.append(bad_conf[i].item())

    # --- METRIC 1: Overall Accuracy ---
    acc = accuracy_score(all_labels, all_preds)
    print(f"\n✅ Top-1 Accuracy: {acc*100:.2f}%")
    
    # --- METRIC 2: Top-K Accuracy ---
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    top_k_hits = 0
    for i in range(len(all_labels)):
        # Get indices of top K probabilities
        top_indices = all_probs[i].argsort()[-top_k:][::-1]
        if all_labels[i] in top_indices:
            top_k_hits += 1
            
    top_k_acc = top_k_hits / len(all_labels)
    print(f"✅ Top-{top_k} Accuracy: {top_k_acc*100:.2f}%")
    
    # --- METRIC 3: Classification Report ---
    class_names = label_encoder.classes_
    print(f"\n📝 Classification Report (Per-Class Performance):")
    print(classification_report(all_labels, all_preds, target_names=class_names, digits=4))
    
    # --- PLOT 1: Confusion Matrix Heatmap ---
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(20, 16))
    sns.heatmap(cm, annot=False, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix Heatmap')
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # --- PLOT 2: Top Confused Pairs ---
    # Find which pairs are most frequently confused
    np.fill_diagonal(cm, 0) # Ignore correct predictions
    pairs = []
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            if cm[i, j] > 0:
                pairs.append((class_names[i], class_names[j], cm[i, j]))
    
    # Sort by count
    pairs.sort(key=lambda x: x[2], reverse=True)
    
    print("\n⚠️ Most Common Confusions (True -> Predicted):")
    for true_l, pred_l, count in pairs[:10]:
        print(f"   • '{true_l}'  mistaken for  '{pred_l}' : {count} times")
        
    # --- PLOT 3: Visual Error Gallery ---
    if len(error_images) > 0:
        print("\n🖼️ Gallery of Mistakes (High Confidence Errors):")
        cols = 5
        rows = min(4, (len(error_images) + cols - 1) // cols)
        plt.figure(figsize=(15, 3*rows))
        
        for i in range(min(len(error_images), 20)):
            plt.subplot(rows, cols, i+1)
            # Un-normalize for display: (img * std) + mean
            img = error_images[i].permute(1, 2, 0).numpy()
            img = (img * 0.224) + 0.456
            img = np.clip(img, 0, 1)
            
            true_name = label_encoder.inverse_transform([error_true[i]])[0]
            pred_name = label_encoder.inverse_transform([error_pred[i]])[0]
            
            plt.imshow(img)
            plt.title(f"True: {true_name}\nPred: {pred_name}\nConf: {error_conf[i]*100:.0f}%", color='red')
            plt.axis('off')
        plt.tight_layout()
        plt.show()

# --- USAGE ---
# evaluate_model_comprehensive(model, val_loader, device, kurdish_dataset.label_encoder)

In [9]:
evaluate_model_comprehensive(model, val_loader, device, kurdish_dataset.label_encoder)

NameError: name 'val_loader' is not defined